# Iteratively Prompting GPT using OpenAI API

In this tutorial, we'll explore how to harness the power of iterative prompting with OpenAI's GPT models. By feeding the chat history back into the model at each turn, we enable it to maintain context throughout the conversation, resulting in more coherent and relevant responses.

### Importing OpenAI API library

This cell imports the OpenAI library into your Python environment. This is necessary to interact with OpenAI's API in your code. The openai library provides the functions and methods needed to send requests to OpenAI's models and receive their responses.

In [1]:
from openai import OpenAI
import os

### Setting up API Key and Creating a Client

This cell initializes the API key and model you will be using. The API key is a unique identifier that allows the OpenAI API to authenticate your requests. Make an API key using this [link](https://platform.openai.com/api-keys). It's essential to keep this key secure. 

In [2]:
# There are several ways to set your API key

# Hard-code your API key
# API_KEY = "..."      # Paste your API key here

# Set an environment variable to hold your API key
# API_KEY = os.getenv('API_KEY')

# Get API_KEY from a file
import json
with open('config.json') as f:
    config = json.load(f)

API_KEY = config['OPENAI_API_KEY']

In [3]:
client = OpenAI(api_key=API_KEY)

### Choosing a Model

The model string specifies which version of the GPT model you intend to use for your requests. Look [here](https://platform.openai.com/docs/models/model-endpoint-compatibility) to check model endpoint compatibility. Make sure the API key and model are correct and valid for your use case. I have chosen a GPT-3.5 Turbo model for this tutorial as these models can understand and generate natural language and have been optimized for chat using the Chat Completions API.

In [4]:
MODEL = "gpt-3.5-turbo-0125"       # 16K context window, optimized for dialog

### Defining a Function with Chat History Argument

Here, we define a function `prompt_gpt` that interacts with OpenAI's chat model. It takes a list of messages as argument, prompts the user for input, appends the user's message to the list, sends the entire conversation history to GPT, retrieves the AI's response, appends it to the list, prints the AI's response, and returns the updated list of messages. If the user inputs "stop", it returns None to signal the end of the conversation. This function facilitates an iterative conversation loop with the GPT-3 model.

In [5]:
def prompt_gpt(messages, system_message):
    # input the user
    user_input = input("You: ")
    
    # break condition
    if user_input.lower() == 'stop':
        return None  # return None to signal the end of the conversation
    
    # append user's dialog to messages
    messages.append({"role": "user", "content": user_input})
    
    # generate the response with history
    response_with_history = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=100,
        temperature=0.0,
        seed=123
    )
    
    # append GPT's dialog to messages (with history)
    messages.append({"role": "system", "content": response_with_history.choices[0].message.content})
    
    # generate the response without history
    response_without_history = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_input}
        ],
        max_tokens=100,
        temperature=0.0,
        seed=123
    )
    
    # print the outputs
    print("GPT (with history): " + response_with_history.choices[0].message.content)
    print("GPT (without history): " + response_without_history.choices[0].message.content + "\n")
    
    return messages

### Iteratively Prompting

With the function defined, we are ready to iteratively prompt GPT. I initialize `messages` with a system role content. This does two things: (i) firstly, it helps constrain the entire conversation and allows you to specify the way the model answers questions, and (ii) secondly, as we continue to converse, the messages (chat history) will continue to grow. This will increase the API credit consumption and might exceed the input token limit. Initializing `messages` here ensures that each time we run the cell and start a conversation, the chat history refreshes.

### Tutorial

In this tutorial, we'll demonstrate the importance of iteratively prompting a large language model (LLM) with conversation history to enhance the quality of its responses. As an example, we'll play a game of physician akinator with GPT. Here, the LLM asks us the most relevant questions to narrow down on what disease we have in mind. Try running the above code cell and successively prompting with the following instructions (feel free to select the right response based on the question asked):

- "I've been experiencing a persistent cough and a mild fever."
- "Yes, I have been in close contact with someone who is sick."
- "Yes, I have shortness of breath and fatigue."
- "Yes, I have a recent loss of taste and smell."

Note that chat completions are non-deterministic by default.  Setting the `seed` parameter may help, but determinism is not guarantted.  For more information, see the [OpenAI API User Guide](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs).

In [6]:
system_message = "You are a physician akinator - a highly skilled doctor trained at diagnosing a patient by repeatedly asking them relevant questions until you are completely sure of the disease. Once you are sure of the diagnosis, you respond with only the name of the disease as <disease>"
# system_message = "You are a physician akinator - a highly skilled doctor trained at diagnosing a patient by repeatedly asking them relevant questions until you are completely sure of the disease. Once you are sure of the diagnosis, you respond with only the name of the disease."
messages = [
    {"role": "system", "content": system_message}
]

# enter "stop" to exit
while True:
    messages = prompt_gpt(messages, system_message)
    if messages is None:
        break

You:  I've been experiencing a persistent cough and a mild fever.


GPT (with history): Have you traveled recently or been in contact with anyone who is sick?
GPT (without history): Have you traveled recently or been in contact with anyone who is sick?



You:  Yes, I have been in close contact with someone who is sick.


GPT (with history): Have you noticed any shortness of breath or chest pain along with the cough and fever?
GPT (without history): Have you experienced any symptoms yourself since being in close contact with the sick individual?



You:  Yes, I have shortness of breath and fatigue.


GPT (with history): Have you had any recent loss of taste or smell?
GPT (without history): Do you have a cough or chest pain along with the shortness of breath and fatigue?



You:  Yes, I have a recent loss of taste and smell.


GPT (with history): COVID-19
GPT (without history): Have you experienced any other symptoms such as fever, cough, or difficulty breathing?



You:  stop
